In [8]:
import sys
import pandas as pd
import gspread
from gspread_pandas import Spread, Client
gc = gspread.service_account()
sys.path.append('/Users/cody/github/CambridgeSemiticsLab/Gesenius_data/tools')
from load_dfs import *

In [9]:
main_semantics = [
    'PAST..IND',     # she wrote
    'PRES.PERF.IND', # he has written
    'PRES..IND',     # she writes
    'PAST.PERF.IND', # she had written
]

# isolate cases of strange qatals
odd_df = qatal_dfs[(~qatal_dfs.esv_TAM.isin(main_semantics)) & qatal_dfs.esv_TAM]

odd_df.head(2)

,ref,book,book_super,canon_part,text_full,text_plain,lex,lex_etcbc,gloss,verb_form,...,niv_tags,niv_VBtags,niv_verse,niv_tense,niv_aspect,niv_modality,niv_TAM,niv_TAMtag,niv_TAMspan,safe
bhsa_node,,,,,,,,,,,,,,,,,,,,,
1189,Genesis 3:1,Genesis,Genesis,Law,אָמַ֣ר,אמר,אמר,>MR[,say,qtl,...,VBD|VB,VBD|VB,Now the serpent was more crafty than any of th...,PRES,NaN,IMPV,PRES..IMPV,IMPV,say,True
1819,Genesis 4:9,Genesis,Genesis,Law,יָדַ֔עְתִּי,ידעתי,ידע,JD<[,know,qtl,...,PRP|VB,VB,"Then the LORD said to Cain , “ Where is your b...",PRES,NaN,IMPV,PRES..IMPV,IMPV,know,True


In [10]:
client = Client(load_dirs=True)

In [11]:
folders = client.find_folders('transitivity')

folder = folders[0]['id']

folder

'1xD5GdNOCqc93MZRFLkwLr12NQURhjJz1'

In [12]:
client.list_spreadsheet_files_in_folder(folder)

[{'id': '1reeZ7RE7oJ1JXnXl0Uo5wdKGgMYTzHiTTTMrOBL3V8g',
  'name': 'intransitive_verbs',
  'path': '/'},
 {'id': '1Sj6o3NXwIQiRUC1T5WBZpKeI8w5pakXbFx1RxnNA7s0',
  'name': 'transitive_verbs',
  'path': '/'}]

In [13]:
def get_existing_sheet(foldername, filename):
    """Retrieve existing Drive sheet if it exists.
    
    Args:
        foldername: str of the folder name
        filename: str of the filename
        
    Returns: 
        if sheet exists, dict of sheet data 
        with keys: id, name, path; otherwise 
        returns None
    """
    
    # NB: avoid double-naming folders in the drive
    # since it is not possible to provide a full path
    folders = client.find_folders(foldername)
    if not folders:
        return None
    
    # search for first matching sheet name
    # within the matched folders
    for folder in folders:
        for sheet in client.list_spreadsheet_files_in_folder(folder['id']):
            if sheet['name'] == filename:
                return sheet
        
    # no match; return a None
    return None

In [14]:
get_existing_sheet('transitivity', 'intransitive_verbs')

{'id': '1reeZ7RE7oJ1JXnXl0Uo5wdKGgMYTzHiTTTMrOBL3V8g',
 'name': 'intransitive_verbs',
 'path': '/'}

In [15]:
tsheet = gc.open_by_key('1reeZ7RE7oJ1JXnXl0Uo5wdKGgMYTzHiTTTMrOBL3V8g')

tsheet

<Spreadsheet 'intransitive_verbs' id:1reeZ7RE7oJ1JXnXl0Uo5wdKGgMYTzHiTTTMrOBL3V8g>

In [17]:
wsheet = tsheet.get_worksheet(0)

wsheet

<Worksheet 'Sheet1' id:0>

In [20]:
wsheet.acell('A3')

<Cell R3C1 'בוא.qal'>

In [29]:
client.directories

[{'id': '141S_J8BpjAxVKJG9LSyHCL-oESIhSWmk', 'name': 'manual'},
 {'id': '1959EfOt8YZgT_rWz6dgU7zNwRGoLzK8P', 'name': 'filter_qatal'},
 {'id': '1xD5GdNOCqc93MZRFLkwLr12NQURhjJz1', 'name': 'transitivity'},
 {'id': '1FN189deeSlcyVXrpDO5twvmqdc_t7HDl', 'name': 'archive'},
 {'id': '1yiROXd5_27ex771giJ3vaV3N-CLKiPTx', 'name': 'datasets'}]

In [31]:
client.

'geseniusdata@gesenius-data.iam.gserviceaccount.com'

In [21]:
client.find_folders('transitivity')

[{'id': '1xD5GdNOCqc93MZRFLkwLr12NQURhjJz1', 'name': 'transitivity'}]

In [42]:
def get_sheet(foldername, filename, exist_ok=True):
    """Retrieves a sheet by foldername and file to perform operations on.
    
    Args:
        foldername: str name of the folder
        filename: str name of the sheet to be saved
    Returns:
        gcspread Spreadsheet object
    """
    
    # retrieve sheet if it already exists
    exist_sheet = get_existing_sheet(foldername, filename)
        
    # load sheet
    if exist_sheet:
        if not exist_ok:
            raise Exception('File already exists! Set exist_ok=True to force.')
        sheet = gc.open_by_key(exist_sheet['id']) 
    else:
        folders = client.find_folders(foldername)
        if not folders:
            raise Exception('Folder does not exist!')
        
        # NB: selects the first available folder with matching name
        # thus you must avoid double-naming folders!! There is no
        # way I know of r/n to do this with a full path.
        folder = folders[0]
        sheet = gc.create(filename, folder_id=folder['id'])
        
    return sheet

def df_to_drive(df, foldername, filename, worksheet_index=0):
    """Write a DataFrame to Google Drive."""
    sh = get_sheet(foldername, filename)    
    spread = Spread(sh.id)
    spread.df_to_sheet(df, replace=True)

In [41]:
odd_df.columns

Index(['ref', 'book', 'book_super', 'canon_part', 'text_full', 'text_plain',
       'lex', 'lex_etcbc', 'gloss', 'verb_form', 'stem', 'person', 'gender',
       'number', 'clause_atom', 'clause', 'sentence', 'genre', 'domain',
       'domain2', 'period', 'txt_type', 'clause_type', 'cltype_simp',
       'clause_rela', 'mother_clause', 'mother_type', 'mother_verbtype',
       'mother_rela', 'mother_domain2', 'daught_clause', 'daught_type',
       'daught_verbtype', 'daught_rela', 'daught_domain2', 'valence',
       'prec_lexes', 'prec_pos', 'prec_part', 'lxx', 'lxx_tense', 'lxx_voice',
       'lxx_mood', 'lxx_person', 'lxx_number', 'lxx_tm', 'esv', 'esv_tags',
       'esv_VBtags', 'esv_verse', 'esv_tense', 'esv_aspect', 'esv_modality',
       'esv_TAM', 'esv_TAMtag', 'esv_TAMspan', 'niv', 'niv_tags', 'niv_VBtags',
       'niv_verse', 'niv_tense', 'niv_aspect', 'niv_modality', 'niv_TAM',
       'niv_TAMtag', 'niv_TAMspan', 'safe'],
      dtype='object')

In [54]:
sample_odd = odd_df[['ref', 'text_full', 'sentence', 'esv_TAM', 'esv_TAMspan', 'niv_TAM', 'niv_TAMspan']]

df_to_drive(sample_odd, 'filter_qatal', 'odd_tam_tags')

In [53]:
list(sample_odd.columns).index('esv_TAM')

3

In [65]:
sh = get_sheet('filter_qatal', 'odd_tam_tags')
worksheet = sh.get_worksheet(0)

In [66]:
worksheet.acell('A3')

<Cell R3C1 '1819'>

In [68]:
cell = worksheet.find('bhsa_node')

cell

<Cell R1C1 'bhsa_node'>

In [70]:
cell.col

1